In [ ]:
import pickle
import os
import pandas as pd
import json

In [ ]:
BIOS_FILE_REVIEWED = "data/BIOS_REVIEWED.pkl" # only reviewed samples
BIOS_FILE_COMPLETE = "data/BIOS_COMPLETE.pkl" # whole dataset with changes to the reviewed samples
BIOS_LABELS = "data/BIOS_LABELS.csv" # this contains all of our changes without the sensitive data, but ids matching the BIOS dataset
BIOS_RAW = "../biosbias/data2023/BIOS.pkl"#"data/BIOS.pkl"
TITLE_JSON = "data/title_lookup.json"

reviewed_classes = ['architect', 'surgeon', 'dentist', 'teacher', 'psychologist', 'nurse', 'photographer', 'physician', 'attorney', 'journalist']

In [ ]:
with (open(BIOS_RAW, "rb")) as openfile:
    raw_data = pickle.load(openfile)

In [ ]:
df_label = pd.read_csv(BIOS_LABELS, sep='\t')
labeled_data = df_label.to_dict('records')

In [ ]:
for i in range(len(labeled_data)):
    if not (raw_data[i]['raw_title'] == labeled_data[i]['auto_raw_title'] and raw_data[i]['start_pos'] == labeled_data[i]['start_pos']):
        print("data mismatch at id ", i)
        
    labeled_data[i]['raw'] = raw_data[i]['raw']
    labeled_data[i]['bio'] = raw_data[i]['bio']
    labeled_data[i]['name'] = raw_data[i]['name']
    
    # convert titles from nan/string to list
    if type(labeled_data[i]['titles']) == float:
        labeled_data[i]['titles'] = []
    elif type(labeled_data[i]['titles']) == str:
        titles = labeled_data[i]['titles'].strip(']"\'[').split('\', \'')
        labeled_data[i]['titles'] = titles
        
    # convert raw titles fron nan/string to list
    if type(labeled_data[i]['raw_titles']) == float:
        labeled_data[i]['raw_titles'] = []
    elif type(labeled_data[i]['titles']) == str:
        titles = labeled_data[i]['raw_titles'].strip(']"\'[').split(', ')
        labeled_data[i]['raw_titles'] = titles

In [ ]:
df = pd.DataFrame(data=labeled_data)
df_reviewed = df[df['review'] == 1]
reviewed_data = df_reviewed.to_dict('records')

In [ ]:
with (open(BIOS_FILE_COMPLETE, "wb")) as openfile:
    pickle.dump(labeled_data, openfile)

with (open(BIOS_FILE_REVIEWED, "wb")) as openfile:
    pickle.dump(reviewed_data, openfile)